In [1]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages
from langchain_ollama import ChatOllama
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.graph import StateGraph, START
from langgraph.checkpoint.memory import MemorySaver

# --- STEP A: Define the State ---
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

# --- STEP B: Define the Tools and Nodes ---
@tool
def search_tool(query: str):
    """Search the web for info using DuckDuckGo."""
    search = DuckDuckGoSearchRun()
    return search.invoke(query)

tools = [search_tool]
tool_node = ToolNode(tools)

model = ChatOllama(model="llama3.2:latest").bind_tools(tools)

def call_model(state: AgentState):
    response = model.invoke(state["messages"])
    return {"messages": [response]}

# --- STEP C: Define the Logic ---
def should_continue(state: AgentState):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return "__end__"

# --- STEP D: Build the Graph with Interrupt ---
workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent")

memory = MemorySaver()

#intterrup_before tools is added.
app = workflow.compile(checkpointer=memory, interrupt_before=["tools"])

The intterupt

In [2]:

config = {"configurable": {"thread_id": "human_testing"}}
input_msg = {"messages": [("user", "Search for the price of Gold in Nepal")]}

# The agent will run, call the LLM, and then STOP before calling the search tool.
for event in app.stream(input_msg, config):
    print(event)

# 2. Check the state
snapshot = app.get_state(config)
print(f"Graph is paused at: {snapshot.next}") # Output: ('tools',)

# 3. Human Approval (Resume)
# Passing None tells the graph to just 'continue' from where it was paused
app.invoke(None, config)

{'agent': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2026-01-13T09:59:28.5784302Z', 'done': True, 'done_reason': 'stop', 'total_duration': 12003330200, 'load_duration': 3902902700, 'prompt_eval_count': 161, 'prompt_eval_duration': 6995689000, 'eval_count': 16, 'eval_duration': 1074949600, 'logprobs': None, 'model_name': 'llama3.2:latest', 'model_provider': 'ollama'}, id='lc_run--019bb6cb-9f5a-76a3-8168-8b38fd807fad-0', tool_calls=[{'name': 'search_tool', 'args': {'query': 'Gold price in Nepal'}, 'id': '096371c7-c191-44d6-b005-59d6ef8baa6c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 161, 'output_tokens': 16, 'total_tokens': 177})]}}
{'__interrupt__': ()}
Graph is paused at: ('tools',)


{'messages': [HumanMessage(content='Search for the price of Gold in Nepal', additional_kwargs={}, response_metadata={}, id='99095818-d3e7-4c12-abc2-9ff3f665ca8a'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2026-01-13T09:59:28.5784302Z', 'done': True, 'done_reason': 'stop', 'total_duration': 12003330200, 'load_duration': 3902902700, 'prompt_eval_count': 161, 'prompt_eval_duration': 6995689000, 'eval_count': 16, 'eval_duration': 1074949600, 'logprobs': None, 'model_name': 'llama3.2:latest', 'model_provider': 'ollama'}, id='lc_run--019bb6cb-9f5a-76a3-8168-8b38fd807fad-0', tool_calls=[{'name': 'search_tool', 'args': {'query': 'Gold price in Nepal'}, 'id': '096371c7-c191-44d6-b005-59d6ef8baa6c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 161, 'output_tokens': 16, 'total_tokens': 177}),
  ToolMessage(content="22 hours ago · LivePriceofGold provides live rates for gold in Nepal , spanning a range of purities includi